In [32]:
#imports
import re

In [65]:
#load small input
input = open("./../../data/day_19/small_input.txt", "r").readlines()

In [89]:
#load input
input = open("./../../data/day_19/input.txt", "r").readlines()

In [90]:
#process input
workflows = {}
parts = []
index_empty_line = input.index("\n")
for line in input[:index_empty_line]:
    line = line.strip()
    res = re.match("(.*)\{(.*)\}",line)
    wf_name = res.group(1)
    wf_rules = res.group(2).split(",")
    rules = []
    for rule in wf_rules:
        res = re.match("([xmas])([<>])([0-9]+):(.*)",rule)
        if res is None:
            if rule == "A":
                rules.append( ("A",))
            elif rule == "R":
                rules.append( ("R",))
            else:
                rules.append( (rule,))
        else:
            r_cat = res.group(1)
            r_op = res.group(2)
            r_val = int(res.group(3))
            r_next = res.group(4)
            rules.append( (r_cat, r_op, r_val, r_next) )
    workflows[wf_name] = rules
for line in input[index_empty_line+1:]:
    line = line.strip()
    res = re.match("\{x=([0-9]*),m=([0-9]*),a=([0-9]*),s=([0-9]*)\}",line)
    part = {
        "x": int(res.group(1)),
        "m": int(res.group(2)),
        "a": int(res.group(3)),
        "s": int(res.group(4))
    }
    parts.append(part)


In [91]:
# compute part 1
accepted = []
for part in parts:
    current_wf_name = "in"
    while True:
        if current_wf_name == "A":
            accepted.append(part)
            break
        elif current_wf_name == "R":
            break
        wf = workflows[current_wf_name]
        for rule in wf:
            if len(rule) == 1: #rule is a default rule
                current_wf_name = rule[0]
            else: #rule is a proper rule
                if rule[1] == "<":
                    if part[rule[0]] < rule[2]:
                        current_wf_name = rule[3]
                        break
                elif rule[1] == ">":
                    if part[rule[0]] > rule[2]:
                        current_wf_name = rule[3]
                        break
                else:
                    print(f"Unexpected operator {rule[1]}")

result = sum([sum(part.values()) for part in accepted])
print(f"Part 1: {result}")


Part 1: 350678


In [92]:
#idea: trace back from all rules that lead to "A" and check what the values need to be for the rule to be satisfied
accept_rules = []
for k,rules in workflows.items():
    for i,rule in enumerate(rules):
        if len(rule) == 1 and rule[0]=="A":
            accept_rules.append( (k,i) )
        elif len(rule) == 4 and rule[3] == "A":
            accept_rules.append( (k,i) )

#compute backlinks
backlinks = {}
for k,rules in workflows.items():
    for i,rule in enumerate(rules):
        if len(rule) == 4:
            link = rule[3]
            if link not in backlinks:
                backlinks[link] = []
            backlinks[link].append( (k,i) )

# compute tree/graph
class Node():
    def __init__(self, name):
        self.name = name
        self.out_connections = []
        self.in_connections = []
    
    def add_out_connection(self, node):
        self.out_connections.append(node)
    
    def add_in_connection(self, node):
        self.in_connections.append(node)

    def get_out_connections(self):
        return self.out_connections
    
    def get_in_connections(self):
        return self.in_connections
    
def get_connection(node_start, node_target):
    out = node_start.get_out_connections()
    r = None
    for node, restrictions in out:
        if node.name == node_target.name:
            if r is not None:
                print(f"Hello there, you have some more cases to consider")
            r = (node, restrictions)
    return r

graph = {}
#instantiate empty nodes
graph["A"] = Node("A")
graph["R"] = Node("R")
for k,rules in workflows.items():
    graph[k] = Node(k)
#add connections
for k,rules in workflows.items():
    for i,rule in enumerate(rules):
        restrictions = []
        for p_rule in rules[:i]:
            restrictions.append( (p_rule[0], ">=" if p_rule[1] == "<" else "<=", p_rule[2]) )
        restrictions = tuple(restrictions)
        if len(rule) == 1:
            if rule[0] == "A":
                graph[k].add_out_connection( (graph["A"],restrictions) )
                graph["A"].add_in_connection( (graph[k],restrictions) )
            elif rule[0] == "R":
                graph[k].add_out_connection( (graph["R"],restrictions) )
                graph["R"].add_in_connection( (graph[k],restrictions) )
            else:
                graph[k].add_out_connection( (graph[rule[0]],restrictions) )
                graph[rule[0]].add_in_connection( (graph[k],restrictions) )
        else:
            restrictions = (*restrictions, (rule[0], rule[1], rule[2]))
            graph[k].add_out_connection( (graph[rule[3]],restrictions) )
            graph[rule[3]].add_in_connection( (graph[k],restrictions) )



In [94]:
#check graph by computing paths through it
##bfs to find all paths
q = []
start_node = graph["in"]
q.append( (start_node, ()) )

paths = []
while len(q) > 0:
    node, prev = q.pop(0)
    if node.name == "A":
        paths.append((*prev,(node,-1)))
        continue
    elif node.name == "R":
        continue
    else:
        for i,(out_node, _) in enumerate(node.get_out_connections()):
            q.append( (out_node, (*prev,(node,i))) )

readable_paths = []
for path in paths:
    readable_paths.append( [node.name for node,index in path] )

r = []
##check conditions for paths
for path in paths:
    path_restrictions = {
        "x": {"min":1, "max":4000},
        "m": {"min":1, "max":4000},
        "a": {"min":1, "max":4000},
        "s": {"min":1, "max":4000}
    }
    for i in range(len(path)-1):
        node,index = path[i]
        next_node,next_index = path[i+1]
        _, restrictions = node.get_out_connections()[index]
        for j,restriction in enumerate(restrictions):
            #print(f"Processing restriction {restriction}")
            if restriction[1] == "<":
                path_restrictions[restriction[0]]["max"] = min(path_restrictions[restriction[0]]["max"], restriction[2]-1)
            elif restriction[1] == ">":
                path_restrictions[restriction[0]]["min"] = max(path_restrictions[restriction[0]]["min"], restriction[2]+1)
            elif restriction[1] == "<=":
                path_restrictions[restriction[0]]["max"] = min(path_restrictions[restriction[0]]["max"], restriction[2])
            elif restriction[1] == ">=":
                path_restrictions[restriction[0]]["min"] = max(path_restrictions[restriction[0]]["min"], restriction[2])
            else:
                print(f"Unexpected operator {restriction[1]}")
            #print(f"->pr: {path_restrictions}")
    r.append(path_restrictions)

#calculate sum
res = 0
for i,t in enumerate(r):

    x_max = t["x"]["max"]
    x_min = t["x"]["min"]
    m_max = t["m"]["max"]
    m_min = t["m"]["min"]
    a_max = t["a"]["max"]
    a_min = t["a"]["min"]
    s_max = t["s"]["max"]
    s_min = t["s"]["min"]


    x = x_max - x_min + 1 if x_max >= x_min else 0
    m = m_max - m_min + 1 if m_max >= m_min else 0
    a = a_max - a_min + 1 if a_max >= a_min else 0
    s = s_max - s_min + 1 if s_max >= s_min else 0   

    add = x*m*a*s

    res += add

## problem: the rating combinations need to be distinct -> currently, the possible combinations per path are calculated but it is not considered if a combination is already on another path

print(f"Part 2: {res}")


Part 2: 124831893423809
